# The Greenhouse Gas (GHS) emissions

In [1]:
# HIDDEN
%load_ext autoreload
%autoreload 2

## Get GHG emission from agricultural land-use

In [2]:
# HIDDEN
import altair as alt

from tools import get_all_files, get_GHG_file_df
from tools.Plot_GHG import get_GHG_plots,plot_GHG_total

## Data preparing

In [3]:
# HIDE CODE
# Get all output files
from PARAMETERS import DATA_ROOT
files = get_all_files(DATA_ROOT)


# Select files based on criteria
files_selc = files.query('year <= 2050')
print(files_selc['catetory'].unique(),'\n')

['GHG' 'cost' 'revenue' 'water' 'cross_table' 'quantity' 'dvar' 'ammap'
 'lmmap' 'lumap'] 



Filter the GHG data

In [4]:
# NO CODE
# Get the GHG files
GHG_files = get_GHG_file_df(files_selc)
GHG_files = GHG_files.reset_index(drop=True).sort_values(['year','GHG_sum_t'])
GHG_files['GHG_sum_Mt'] = GHG_files['GHG_sum_t'] / 1e6
GHG_files

,year,year_types,catetory,base_name,base_ext,path,GHG_sum_t,GHG_sum_Mt
2,2010,single_year,GHG,Non-Agricultural Landuse,.csv,../docker_data/LUTO_DATA/CWC15_2023_11_23__07_...,-1.077782e+05,-0.107778
1,2010,single_year,GHG,Agricultural Management,.csv,../docker_data/LUTO_DATA/CWC15_2023_11_23__07_...,0.000000e+00,0.000000
3,2010,single_year,GHG,Transition Penalty,.csv,../docker_data/LUTO_DATA/CWC15_2023_11_23__07_...,0.000000e+00,0.000000
0,2010,single_year,GHG,Agricultural Landuse,.csv,../docker_data/LUTO_DATA/CWC15_2023_11_23__07_...,9.051190e+07,90.511903
5,2011,single_year,GHG,Agricultural Management,.csv,../docker_data/LUTO_DATA/CWC15_2023_11_23__07_...,-7.020405e+05,-0.702040
...,...,...,...,...,...,...,...,...
156,2049,single_year,GHG,Agricultural Landuse,.csv,../docker_data/LUTO_DATA/CWC15_2023_11_23__07_...,7.678331e+07,76.783311
162,2050,single_year,GHG,Non-Agricultural Landuse,.csv,../docker_data/LUTO_DATA/CWC15_2023_11_23__07_...,-3.156811e+08,-315.681060
161,2050,single_year,GHG,Agricultural Management,.csv,../docker_data/LUTO_DATA/CWC15_2023_11_23__07_...,-2.598144e+07,-25.981440
163,2050,single_year,GHG,Transition Penalty,.csv,../docker_data/LUTO_DATA/CWC15_2023_11_23__07_...,6.538516e+04,0.065385


Calculate the net emissions

In [5]:
# HIDDEN
# Calculate the net emissions
Net_emission = GHG_files.groupby('year')['GHG_sum_Mt'].sum(numeric_only = True).reset_index()
Net_emission = Net_emission.rename(columns={'GHG_sum_Mt':'Net_emission'})

Net_emission['Net_emission_cum'] = Net_emission['Net_emission'].cumsum()
Net_emission.head(3)

,year,Net_emission,Net_emission_cum
0,2010,90.404125,90.404125
1,2011,88.270862,178.674987
2,2012,86.541724,265.216711


## Total GHG emissions across years

In [6]:
# HIDDEN
cumsum_plot, individual_plot = plot_GHG_total(GHG_files, Net_emission)

In [7]:
# HIDE CODE 
cumsum_plot

alt.Chart(...)

In [8]:
# HIDE CODE 
individual_plot

alt.LayerChart(...)

## Agricultural Land use Emissions

Get the Agricultural land-use data

In [9]:
# HIDDEN
# Initialize the class
ag_lucc_GHG = get_GHG_plots(GHG_files,'Agricultural Landuse','GHG emissions (Mt CO2e)')

### Emissions by Crops/Livestocks

In [10]:
# HIDE CODE
# get the df and plot
GHG_crop_lvstk_df,GHG_crop_lvstk_plot = ag_lucc_GHG.plot_GHG_crop_lvstk()
GHG_crop_lvstk_plot

alt.LayerChart(...)

### Emissions by differrnt irrigation system

In [11]:
# HIDE CODE
# get the df and plot
GHG_dry_irr_df,GHG_dry_irr_plot = ag_lucc_GHG.plot_GHG_dry_irr()
GHG_dry_irr_plot

alt.LayerChart(...)

### Emissions by GHG category

In [12]:
# HIDE CODE
# get the df and plot
GHG_catetory_df,GHG_atetory_plot = ag_lucc_GHG.plot_GHG_category()
GHG_atetory_plot

alt.LayerChart(...)

### Emission by CO2 sources

In [24]:
# HIDE CODE
# get the df and plot
GHG_sources_df,GHG_sources_plot = ag_lucc_GHG.plot_GHG_sources()
GHG_sources_plot

alt.LayerChart(...)

### Emissions by {land-use}-{irrigation} cmobined

In [14]:
# HIDE CODE
# get the df and plot
GHG_lu_lm_df,GHG_lu_lm_plot = ag_lucc_GHG.plot_GHG_lu_lm(2050)
GHG_lu_lm_plot

alt.Chart(...)

### Emissions by {Land-use}-{GHG source} combined

In [15]:
# HIDE CODE
# get the df and plot
GHG_lu_source_df,GHG_lu_source_plot = ag_lucc_GHG.plot_GHG_lu_source(2050)
GHG_lu_source_plot

alt.Chart(...)

## Non-Agricultural Land-use GHG reductions/sequestrations

### Data preparation

In [16]:
# HIDDEN
# Initialize the class
non_ag_lucc_GHG = get_GHG_plots(GHG_files,'Non-Agricultural Landuse','GHG sequestration (Mt CO2e)')

/home/jinzhu/VIS_LUTO/tools/Plot_GHG.py:230: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([GHG_CO2,GHG_nonCO2],axis=0).reset_index(drop=True)


### Sequestrations in total

In [17]:
# HIDE CODE
# get the df and plot
GHG_non_ag_crop_lvstk_df,GHG_non_ag_crop_lvstk_plot = non_ag_lucc_GHG.plot_GHG_crop_lvstk()
GHG_non_ag_crop_lvstk_plot

alt.LayerChart(...)

### Sequestrations by irrigation system

In [18]:
# # HIDE CODE
# # get the df and plot
# GHG_non_ag_dry_irr_df,GHG_non_ag_dry_irr_plot = non_ag_lucc_GHG.plot_GHG_dry_irr()
# GHG_non_ag_dry_irr_plot

## Agricultural Management GHG reductions

### Data preparation

In [19]:
# HIDDEN
# Initialize the class
ag_man_GHG = get_GHG_plots(GHG_files,'Agricultural Management','GHG reductions (Mt CO2e)')

### Sequestrations by Crops/Livestocks

In [20]:
# HIDE CODE
# get the df and plot
GHG_ag_man_GHG_crop_lvstk_df,GHG_ag_man_GHG_crop_lvstk_plot = ag_man_GHG.plot_GHG_crop_lvstk()
GHG_ag_man_GHG_crop_lvstk_plot

alt.LayerChart(...)

### Reductions by irrigation system

In [21]:
# HIDE CODE
# get the df and plot
GHG_ag_man_dry_irr_df,GHG_ag_man_dry_irr_plot = ag_man_GHG.plot_GHG_dry_irr()
GHG_ag_man_dry_irr_plot

alt.LayerChart(...)

### Sequestrations by GHG category

In [22]:
# HIDE CODE
# get the df and plot
GHG_ag_man_df,GHG_ag_man_plot = ag_man_GHG.plot_GHG_category()
GHG_ag_man_plot

alt.LayerChart(...)